In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

import re

In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
##############
# Parameters #
##############

min_gram = 1
max_gram = 3

# To create ours partitions, we must first know the years which will be the limits
limit_years = [2007, 2010, 2014]

# Ignore words that appear at a frequency less than tresh_freq in the corpus
tresh_freq = 0.8

# Number of clusters by partitions
nb_clusters = 5


In [4]:
# Datas preprocessing methods.

# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw

# Based on ration of french and english stopwords
def isEnglish(article):
    total_fsw = len([x for x in article.split() if x in sourceFST])
    total_esw = len([x for x in article.split() if x in stopwords.words('english')])
    ratio = 100
    if total_fsw != 0:
        ratio = total_esw/total_fsw
    return ratio > 1 and total_esw > 3

def lemmatize(article):
    artiregex = re.sub(" [A-z][A-z] ", " ", article) # word of length < 2
    artiregex = artiregex.lower()
    artiregex = re.sub("(é|è|ê)", "e", artiregex)
    output = []
    outPonc = artiregex.translate(artiregex.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST and [x for x in token.lemma_ if x not in "0123456789"] != []:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [5]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [13]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    #if not null, empty, or whatever (so if there is a abstract):
    if not isinstance(data['abstract'][i], float) and not isEnglish(data['abstract'][i]):
        text = data['abstract'][i]
        if not isinstance(data['title'][i], float):
            text += " "+data['title'][i]

        numArti+=1
        usable.append(stemmer.stem(lemmatize(text)))
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti)
limits.append(numArti)


In [7]:
# Display pre-processed datas

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram), max_df=tresh_freq)
tfidf = vectorizer.fit_transform(usable)

print("nombre d'articles =", len(usable))
print("nombre de mots =", len(tfidf.toarray()[0]))
print("limits =", limits)

nombre d'articles = 991
nombre de mots = 133088
limits = [223, 468, 694, 991]


In [41]:
# Creation of partitions_tfidf[], which give us the TFIDF of each cluster of each partition
# partitions_tfidf[num_partition][num_doc][num_word]
# Beware, num_doc can't be equals to 1091 (max). You have partitions, so every doc aren't in every partitions
# num_word can be found via vectorizer.get_feature_name()
partitions_tfidf = []
beg = 0
for l in limits:
    last = l
    partitions_tfidf.append([list(x) for x in list(tfidf.toarray())[beg:last]])
    beg = l

In [58]:
size = len(partitions_tfidf[0]) + len(partitions_tfidf[1]) + len(partitions_tfidf[2]) + len(partitions_tfidf[3])
size

991

In [75]:
len(partitions_tfidf[0])

223

In [14]:
# Applying KMeans on tfidf
# the labels_ give assignment of doc to the cluster number 
km = KMeans(n_clusters=nb_clusters)
km.fit(tfidf)
cluster = km.labels_

cluster_partition = [cluster[:limits[0]],cluster[limits[0]:limits[1]],cluster[limits[1]:limits[2]],cluster[limits[2]:limits[3]]]

In [71]:
part_km = []
for i in range(0, len(limits)):
    dash = km.fit(partitions_tfidf[i])
    part_km.append(dash)

In [76]:
(part_km[1]).labels_

all_labels = []
for i in range(0,len(limits)):
    for j in range(0, len((part_km[i]).labels_)):
        

3

In [66]:
# doc_clustering is a dictionnary 
# it looks like -> { doc_number : [partition_number, cluster_number] }
# This is used to reassign doc number to their respective partition and and cluster
doc_clustering = {}
for i in range(0,len(usable)):
    if i < limits[0]:
        doc_clustering[i] = [0, cluster[i]]
    elif i >= limits[0] and i < limits[1]:
        doc_clustering[i] = [1, cluster[i]]
    elif i >= limits[1] and i < limits[2]:
        doc_clustering[i] = [2, cluster[i]]
    else:
        doc_clustering[i] = [3, cluster[i]]

In [23]:
# Allows to get list of documents number
# return [dou numbers]
# params : partition_number , cluster number
partitions = []
def get_doc(part, clust):
    docs = []
    for i in range(0,len(doc_clustering)):
        if doc_clustering[i][0] == part and doc_clustering[i][1] == clust:
            docs.append(i)
    return docs

In [115]:
# Get the partitions variable
# Here partitions[part][cluster] = list of docs numbe
partitions = []
for i in range(0, len(limits)):
    clusters = []
    for j in range(0, nb_clusters):
        clusters.append(get_doc(i,j))
    partitions.append(clusters)

[700,
 702,
 706,
 707,
 712,
 713,
 715,
 717,
 734,
 736,
 740,
 741,
 744,
 747,
 762,
 764,
 768,
 769,
 781,
 788,
 791,
 792,
 795,
 797,
 806,
 813,
 816,
 822,
 824,
 826,
 827,
 828,
 831,
 839,
 845,
 853,
 855,
 859,
 861,
 866,
 871,
 874,
 878,
 887,
 890,
 900,
 901,
 904,
 910,
 915,
 916,
 917,
 921,
 924,
 926,
 929,
 930,
 934,
 935,
 939,
 946,
 966,
 967,
 978,
 983,
 984,
 990]

In [24]:
# example of output for doc_clustering
# doc 465 is in cluster 1 of the partition 1
# doc 154 is in cluster 2 of the partition 0

print(doc_clustering[465])
print(doc_clustering[154])
print()

# Here, just count docs number by cluster.
print(Counter(cluster_partition[0]))
print(Counter(cluster_partition[1]))
print(Counter(cluster_partition[2]))
print(Counter(cluster_partition[3]))

[1, 3]
[0, 3]

Counter({4: 55, 0: 47, 2: 43, 1: 39, 3: 39})
Counter({0: 62, 3: 54, 2: 46, 1: 43, 4: 40})
Counter({3: 86, 0: 40, 1: 38, 2: 33, 4: 29})
Counter({3: 96, 0: 67, 2: 52, 1: 42, 4: 40})


###### A quoi sert le reste en dessous là ? 

In [ ]:
# num_partition = 0
# for partition in partitions_tfidf:
#     if (num_partition > 2):
#         break
#     print("Partition numéro:",num_partition)
#     num_partition+=1
#     i=0
#     for cluster_tfidf in partition:
#         if (i<3):
#             print("cluster number ", i)
#             i+=1
#             print(cluster_tfidf)


In [ ]:
# # Let's create ours partitions
# partitions = []

# for part in partitions_tfidf:
#     # Let's create ours clusters
#     clusters = []
#     kmeans = KMeans(n_clusters=nb_clusters, random_state=0).fit(part)
#     partitions.append(clusters)
    
# # TODO: it would be nice to know how many articles are in no cluster anymore

In [ ]:
# # Display clusters 3 of partition 0 
# len(partitions[0])

In [ ]:
# nb_labels_by_cluster = 5

# # Let's labelize our clusters
# # For this, we will use the tfidf matrix

# vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram))
# tfidf = vectorizer.fit_transform(usable)

# # We can access the value in the tfidf using:
# #tfidf.toarray().item(num_doc, num_word)
# # To know the number of the word searched, we will use:
# #vectorizer.vocabulary_[word]

# # take less than 8h to compute x)
# labels = []
# for clusters in partitions:
#     l = []
#     for clus in clusters:
#         first_arti = True
#         for article in clus:
#             link = abs(clus[article])
#             if first_arti:
#                 coef_list = (tfidf.toarray()[article] * link)
#                 first = False
#             else:
#                 # the more an article have a high coeficient, the more he is implied in the labeling step
#                 coef_list += (tfidf.toarray()[article] * link)
#         # Now we have coef_list filled by every coeficient in the multiple tfidf
#         # Let's find the best ones, to finally get the labels
#         res = dict(zip(vectorizer.get_feature_names(), coef_list))

#         l.append(Counter(res).most_common(nb_labels_by_cluster))
#     labels.append(l)

# # TODO: on observe beaucoup de labels identiques entre deux clusters
# # Je pense que c'est parce que l'on a trop de clusters, mais j'aimerais en être sûr

In [ ]:
# # Display labels
# # labels is composed by an array for each partition
# labels